In [41]:
#hide
# default_exp bom
from nbdev.showdoc import *

In [42]:
#export
import tempfile
import subprocess
import sys
import os

from fastcore.script import *
import pandas as pd
from kicad_helpers import setup_test_repo, get_project_name, get_schematic_path, get_bom_path
from kicad_helpers.core import _set_root

In [43]:
#hide
def install_python_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install_python_package("git+https://github.com/ryanfobel/KiField.git@add-quantity")
root = tempfile.mkdtemp()
setup_test_repo(root)

  Cloning https://github.com/ryanfobel/KiField.git (to revision add-quantity) to /tmp/pip-req-build-wfwqyapj


  Running command git clone --filter=blob:none -q https://github.com/ryanfobel/KiField.git /tmp/pip-req-build-wfwqyapj
  Running command git checkout -b add-quantity --track origin/add-quantity
  Switched to a new branch 'add-quantity'
  Branch 'add-quantity' set up to track remote branch 'add-quantity' from 'origin'.


  Resolved https://github.com/ryanfobel/KiField.git to commit 88658fe8523acec58cff7ac8bc1c99e3ee126a48
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


Cloning into '/tmp/tmpp6aso6zq'...


In [36]:
#export
@call_parse
def sch_to_bom(v:Param("verbose", bool),
               overwrite:Param("update existing schematic", bool),
               root:Param("project root directory", str)="."):
    """
    Update/create BOM from KiCad schematic.
    """
    root = _set_root(root)
    subprocess.check_output(f"{ sys.executable } -m kifield --nobackup --overwrite --group -aq -x { get_schematic_path(root) } -i { get_bom_path(root) }", shell=True)
    
@call_parse
def bom_to_sch(v:Param("verbose", bool),
               overwrite:Param("update existing schematic", bool),
               root:Param("project root directory", str)="."):
    """
    Update KiCad schematic from BOM file.
    """
    root = _set_root(root)
    subprocess.check_output(f"{ sys.executable } -m kifield --nobackup --overwrite --fields ~quantity -x { get_bom_path(root) } -i { get_schematic_path(root) }", shell=True)

In [33]:
extract_bom_from_schematic(root)
df = pd.read_csv(get_bom_path(root))
df

,Refs,Quantity,MPN,Manufacturer,datasheet,footprint,value
0,"C1, C3, C5, C6, C8-C13, C16",11,CL21B104KBCNNNC,Samsung,NaN,Capacitors_SMD:C_0805,0.1uF
1,"C14, C15",2,CL21C151JBANNNC,Samsung,NaN,Capacitors_SMD:C_0805,150pF
2,"C2, C4",2,T491D336K020AT,KEMET,NaN,Sci-Bots:SM2917,33uF
3,C7,1,CL21B103KCANNNC,Samsung,NaN,Capacitors_SMD:C_0805,0.01uF
4,DS1,1,150080BS75000,Würth Elektronik,NaN,LEDs:LED_0805,+3.3V_PWR
5,FB1-FB5,5,742792040,Würth Elektronik,NaN,Resistors_SMD:R_0805,FERRITE
6,JP1,1,NaN,NaN,NaN,Resistors_SMD:R_0805,JUMPER
7,P1,1,NaN,NaN,NaN,Pin_Headers:Pin_Header_Angled_1x06,CONN_01X06
8,P2,1,NaN,NaN,NaN,Pin_Headers:Pin_Header_Straight_2x03,CONN_01X06
9,P3-P6,4,NaN,NaN,NaN,Connect:1pin,CONN_01X01


In [34]:
update_schematic_from_bom(root)